#### Simplest starter kernel

In [16]:
!nvidia-smi

Wed Mar 18 20:44:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.66       Driver Version: 441.66       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070   WDDM  | 00000000:01:00.0  On |                  N/A |
| 29%   41C    P8    10W / 151W |   7358MiB /  8192MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import numpy as np
import pandas as pd
import os
import json
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)
print(tf.keras.__version__)

C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-pack

1.13.1
2.2.4-tf


In [2]:
!pip install efficientnet
import efficientnet.tfkeras as efn

In [3]:
path='data/'
train_df=pd.read_csv(path + 'train.csv')
test_df=pd.read_csv(path + 'test.csv')

train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [4]:
pat_dic = {'healthy':0, 'scab':1, 'rust':2, 'multiple_diseases':3 }
train_df['all'] = (train_df.iloc[:, 1:] == 1).idxmax(1)
train_df["all"] = train_df["all"].replace(pat_dic).astype('str')

train_df['image_id'] = train_df['image_id'] + '.jpg'

train_df.head()

,image_id,healthy,multiple_diseases,rust,scab,all
0,Train_0.jpg,0,0,0,1,1
1,Train_1.jpg,0,1,0,0,3
2,Train_2.jpg,1,0,0,0,0
3,Train_3.jpg,0,0,1,0,2
4,Train_4.jpg,1,0,0,0,0


In [5]:
nb_classes = 4
batch_size = 64
img_size = 224
nb_epochs = 20

In [6]:
%%time
train_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_dataframe(    
    dataframe=train_df,    
    directory=path + '/images/',
    x_col="image_id",
    y_col="all",
    batch_size=batch_size,
    shuffle=True,
    classes = [str(i) for i in range(nb_classes)],
    class_mode="categorical",    
    target_size=(img_size,img_size))

Found 1821 validated image filenames belonging to 4 classes.
Wall time: 90.8 ms


In [7]:
def get_model():
    base_model =  efn.EfficientNetB2(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

In [8]:
model = get_model()
model.compile(optimizers.RMSprop(lr=0.003, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
31940608/31936256 [==============================] - 2s 0us/step


In [9]:
%%time
model.fit_generator(generator=train_generator,                                                                                                             
                    epochs=nb_epochs,
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[64,24,56,56] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block2c_project_bn/cond/FusedBatchNorm}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node ConstantFoldingCtrl/loss/dense_loss/broadcast_weights/assert_broadcastable/AssertGuard/Switch_0}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [10]:
test_df['image_id'] = test_df['image_id'] + '.jpg'
test_df.head()

,image_id
0,Test_0.jpg
1,Test_1.jpg
2,Test_2.jpg
3,Test_3.jpg
4,Test_4.jpg


In [11]:
%%time
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(      
    
        dataframe=test_df,    
    
        directory = path + '/images/',    
        x_col="image_id",
        target_size = (img_size,img_size),
        batch_size = 1,
        classes = [str(i) for i in range(nb_classes)],
        shuffle = False,
        class_mode = None
        )

C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:211: UserWarning: `classes` will be ignored given the class_mode="None"
  .format(self.class_mode))


Found 1821 validated image filenames.
Wall time: 265 ms


In [12]:
%%time
test_generator.reset()
predict=model.predict_generator(test_generator)
results = np.argmax(predict,axis = 1)

Wall time: 32.4 s


In [13]:
sub_df=pd.read_csv(path + 'sample_submission.csv')
sub_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [14]:
sub_df["all"] = results
sub_df["all"] = sub_df["all"].replace({v: k for k, v in pat_dic.items()})

sub_df.drop([*pat_dic], axis=1, inplace=True)
sub_df = pd.concat([sub_df,pd.get_dummies(sub_df['all'])],axis=1)
sub_df.drop(['all'],axis=1, inplace=True)

sub_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,1,0,0,0
1,Test_1,1,0,0,0
2,Test_2,0,0,0,1
3,Test_3,0,1,0,0
4,Test_4,1,0,0,0


In [15]:
sub_df.to_csv("submission.csv",index=False)